# Домашняя работа

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [91]:
import pandas as pd

In [92]:
df = pd.read_csv("ds_salaries.csv")

In [93]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [94]:
# Подготовка данных - удаление столбца salary, а так же one-hot кодирование для категориальных признаков.
from sklearn.preprocessing import OneHotEncoder

df = df.drop('salary', axis=1)

categorical_features = ['experience_level', 'employment_type', 'job_title', 'salary_currency', 'employee_residence', 'company_location', 'company_size']

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(df[categorical_features])

new_columns = encoder.get_feature_names_out(categorical_features)
encoded_df = pd.DataFrame(encoded_data, columns=new_columns)
data = pd.concat([df.drop(categorical_features, axis=1), encoded_df], axis=1)

data.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,work_year,salary_in_usd,remote_ratio,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,...,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_L,company_size_M,company_size_S
0,2023,85847,100,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2023,30000,100,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2023,25500,100,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,2023,175000,100,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2023,120000,100,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [95]:
from sklearn.model_selection import train_test_split

X = data.drop('salary_in_usd', axis=1)
y = data['salary_in_usd']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.111, random_state=42)

print("Train set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Train set shape: (3003, 276) (3003,)
Validation set shape: (376, 276) (376,)
Test set shape: (376, 276) (376,)


## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [96]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import numpy as np

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print("MAPE:", mape)
print("RMSE:", rmse)

MAPE: 16337976.492665753
RMSE: 9297074634264.207


## Задание 3 (0.5 балла)

In [97]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [98]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import numpy as np

model = xgb.train({
    'objective': 'reg:squarederror',
    'eval_metric': 'mae'
}, xgb.DMatrix(X_train, label=y_train))

y_pred = model.predict(xgb.DMatrix(X_val))

mape = mean_absolute_percentage_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print("MAPE:", mape)
print("RMSE:", rmse)

MAPE: 0.3713625301075384
RMSE: 50914.269232040286


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [99]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder, CountEncoder

data_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

target_encoder = TargetEncoder()
data_encoded = target_encoder.fit_transform(data_encoded, data['salary_in_usd'])

count_encoder = CountEncoder()
data_encoded = count_encoder.fit_transform(data_encoded)

data_encoded.head()

,work_year,salary_in_usd,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title_AI Developer,...,company_location_SG,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_M,company_size_S
0,2023,85847,100,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023,30000,100,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
2,2023,25500,100,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,2023,175000,100,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2023,120000,100,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [101]:
X = data_encoded.drop('salary_in_usd', axis=1)
y = data_encoded['salary_in_usd']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.111, random_state=42)

print("Train set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

Train set shape: (3003, 269) (3003,)
Validation set shape: (376, 269) (376,)
Test set shape: (376, 269) (376,)


In [102]:
from sklearn.ensemble import GradientBoostingRegressor

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

linear_predictions = linear_model.predict(X_test)
linear_mape = mean_absolute_percentage_error(y_test, linear_predictions)
linear_rmse = np.sqrt(mean_squared_error(y_test, linear_predictions))


boosting_model = GradientBoostingRegressor()
boosting_model.fit(X_train, y_train)


boosting_predictions = boosting_model.predict(X_test)
boosting_mape = mean_absolute_percentage_error(y_test, boosting_predictions)
boosting_rmse = np.sqrt(mean_squared_error(y_test, boosting_predictions))

print("Линейная модель:")
print("MAPE:", linear_mape)
print("RMSE:", linear_rmse)
print()
print("Модель бустинга:")
print("MAPE:", boosting_mape)
print("RMSE:", boosting_rmse)

Линейная модель:
MAPE: 70080194.84462202
RMSE: 50160800901292.12

Модель бустинга:
MAPE: 0.41774876668845173
RMSE: 50281.29832286191


По полученным данным, очевидно, что модель бустинга работает намного лучше чем линейная модель, так как и MAPE и RMSE у бустинга значительно меньше. Получается, в дальнейшем будем использовать именно модель бустинга.

## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [103]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': boosting_predictions})

results['Error'] = abs(results['Actual'] - results['Predicted'])
top_20_errors = results.sort_values('Error', ascending=False).head(20)

top_20_errors

,Actual,Predicted,Error
1288,385000,128624.211319,256375.788681
1105,370000,163829.286057,206170.713943
33,342810,177368.197711,165441.802289
1116,323300,173708.394457,149591.605543
325,280700,145980.576092,134719.423908
3387,25000,158563.121672,133563.121672
1677,297300,163829.286057,133470.713943
478,353200,222688.886629,130511.113371
358,304000,173708.394457,130291.605543
2009,255000,132556.594816,122443.405184


In [104]:
error_indexes = top_20_errors.index
error_examples = X.loc[error_indexes]
error_examples.to_excel('error_examples.xlsx', index=False)
error_examples

,work_year,remote_ratio,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_FL,employment_type_FT,employment_type_PT,job_title_AI Developer,job_title_AI Programmer,...,company_location_SG,company_location_SI,company_location_SK,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_size_M,company_size_S
1288,2023,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1105,2023,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
33,2023,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1116,2023,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
325,2023,100,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
3387,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1677,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
478,2023,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
358,2023,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2009,2022,100,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


По харакетру данных, можно увидеть закономернность, что топ-20 с самыми большими ошибками, все имеют 1 в столбце employment_type_FT, большинство имеет 1 в столбце experience_level_SE, большинство имеет 1 в столбце company_size_M, большинтво имеет 1 в столбце company_location_US, большинство имеет 1 в столбце salary_currency_USD. Можно сделать вывод, что модель делает наибольшие ошибки в предсказании зарплаты для работников-сеньоров, работающих полный день в США в компании среднего размера и получающих зарплату в долларах.  
  

Решением проблемы вижу следующий способ: поробовать выделить новые столбцы на основании значений в данныз 5-ти подозрительных столбцов.

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

ГИПОТЕЗА  
Попробуем выделить единицы в столбцах employment_type_FT, experience_level_SE, company_size_M, company_location_US, salary_currency_USD, так как если в этих столбцах у элемента стоят 1-цы, то по предыдущему пункут можно предположить, что модель ошибеться и довольно сильно.  
  
  Следовательно, выделим отдельный столбец, в котором будет стоять 1 - если во всех 5-ти данных столбцах у элемента стоят единицы и 0, если не во всех столбцах стоят единицы.

In [105]:
X_train['all_ones'] = (X_train['employment_type_FT'] & X_train['experience_level_SE'] & X_train['company_size_M'] & X_train['company_location_US'] & X_train['salary_currency_USD']).astype(int)

boosting_model_new = GradientBoostingRegressor()
boosting_model_new.fit(X_train, y_train)

boosting_predictions_new = boosting_model.predict(X_test)

boosting_mape_new = mean_absolute_percentage_error(y_test, boosting_predictions_new)
boosting_rmse_new = np.sqrt(mean_squared_error(y_test, boosting_predictions_new))

print("Старая модель бустинга:")
print("MAPE:", boosting_mape)
print("RMSE:", boosting_rmse)
print()
print("Новая модель бустинга:")
print("MAPE:", boosting_mape_new)
print("RMSE:", boosting_rmse_new)

Старая модель бустинга:
MAPE: 0.41774876668845173
RMSE: 50281.29832286191

Новая модель бустинга:
MAPE: 0.41774876668845173
RMSE: 50281.29832286191


Как видно, идея не увенчалась успехом, модели никак не помогло выделение в отдельный столбец "красного флага".

ГИПОТЕЗА  
Теперь попробуем создаь новый столбец, который содержит 1, если в хотя бы одном из подозрительных столбцов есть 1, иначе 0.

In [106]:
data_encoded['any_ones'] = (data_encoded[['employment_type_FT', 'experience_level_SE', 'company_size_M', 'company_location_US', 'salary_currency_USD']] == 1).any(axis=1).astype(int)

boosting_model_new = GradientBoostingRegressor()
boosting_model_new.fit(X_train, y_train)

boosting_predictions_new = boosting_model.predict(X_test)

boosting_mape_new = mean_absolute_percentage_error(y_test, boosting_predictions_new)
boosting_rmse_new = np.sqrt(mean_squared_error(y_test, boosting_predictions_new))

print("Старая модель бустинга:")
print("MAPE:", boosting_mape)
print("RMSE:", boosting_rmse)
print()
print("Новая модель бустинга:")
print("MAPE:", boosting_mape_new)
print("RMSE:", boosting_rmse_new)

Старая модель бустинга:
MAPE: 0.41774876668845173
RMSE: 50281.29832286191

Новая модель бустинга:
MAPE: 0.41774876668845173
RMSE: 50281.29832286191


Точно как и в предыдущей гипотезе, это никак непомогло модели.

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [107]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [114]:
import catboost as cb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

X = df.drop('salary_in_usd', axis=1)
y = df['salary_in_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = cb.CatBoostRegressor(loss_function='RMSE')

params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5]
}

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train, cat_features=categorical_features)

best_params = grid_search.best_params_

model = cb.CatBoostRegressor(loss_function='RMSE', **best_params)
model.fit(X_train, y_train, cat_features=categorical_features)

y_pred = model.predict(X_test)

mape_cat = mean_absolute_percentage_error(y_test, y_pred)
rmse_cat = mean_squared_error(y_test, y_pred, squared=False)

training_time_cat = grid_search.cv_results_['mean_fit_time'].mean()
prediction_time_cat = grid_search.cv_results_['mean_score_time'].mean()

Выходные данные были обрезаны до нескольких последних строк (5000).
200:	learn: 32002.2044559	total: 2.37s	remaining: 1.17s
201:	learn: 31986.6771648	total: 2.39s	remaining: 1.16s
202:	learn: 31975.0818466	total: 2.4s	remaining: 1.15s
203:	learn: 31963.4123954	total: 2.41s	remaining: 1.13s
204:	learn: 31956.8182860	total: 2.42s	remaining: 1.12s
205:	learn: 31924.4965079	total: 2.43s	remaining: 1.11s
206:	learn: 31912.3839152	total: 2.44s	remaining: 1.1s
207:	learn: 31845.4937083	total: 2.46s	remaining: 1.09s
208:	learn: 31803.2738557	total: 2.47s	remaining: 1.08s
209:	learn: 31783.9332574	total: 2.48s	remaining: 1.06s
210:	learn: 31774.6789400	total: 2.49s	remaining: 1.05s
211:	learn: 31770.4696382	total: 2.5s	remaining: 1.04s
212:	learn: 31762.3171616	total: 2.51s	remaining: 1.03s
213:	learn: 31744.4557123	total: 2.52s	remaining: 1.01s
214:	learn: 31737.9634090	total: 2.54s	remaining: 1s
215:	learn: 31692.8721327	total: 2.55s	remaining: 992ms
216:	learn: 31668.1002416	total: 2.57s	rem

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [115]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])

X = df.drop('salary_in_usd', axis=1)
y = df['salary_in_usd']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [4, 6, 8],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1]
}

model = xgb.XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mape_xgb = mean_absolute_percentage_error(y_test, y_pred)
rmse_xgb = mean_squared_error(y_test, y_pred, squared=False)

training_time_xgb = grid_search.cv_results_['mean_fit_time'].mean()
prediction_time_xgb = grid_search.cv_results_['mean_score_time'].mean()

## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [113]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [116]:
import pandas as pd
import lightgbm as lgb

params = {
    'num_leaves': [20, 30, 40],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [4, 6, 8],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [0, 0.1, 1]
}

model = lgb.LGBMRegressor(objective='regression')

grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

model = lgb.LGBMRegressor(objective='regression', **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mape_lgb = mean_absolute_percentage_error(y_test, y_pred)
rmse_lgb = mean_squared_error(y_test, y_pred, squared=False)

training_time_lgb = grid_search.cv_results_['mean_fit_time'].mean()
prediction_time_lgb = grid_search.cv_results_['mean_score_time'].mean()

## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

In [117]:
models = ['cat', 'lgb', 'xgb']
metrics = ['MAPE', 'RMSE', 'Training time', 'Prediction time']
values = [
    [mape_cat, rmse_cat, training_time_cat, prediction_time_cat],
    [mape_lgb, rmse_lgb, training_time_lgb, prediction_time_lgb],
    [mape_xgb, rmse_xgb, training_time_xgb, prediction_time_xgb]
]

data = pd.DataFrame(values, index=models, columns=metrics)
data

,MAPE,RMSE,Training time,Prediction time
cat,0.378717,47577.661397,1.487285,0.004784
lgb,0.387127,48147.481608,0.048201,0.003941
xgb,0.382685,48155.151552,1.088285,0.010504


По полученным результатам помжно увидеть, что с catboost удалось получить наилучшую модель по качеству, т.к. MAPE и RMSE у этой модели минимальные, но и обучалось она дольше всего. Лучшей по скорости обучения и по скорости предсказания оказалась модель, полученная с помощью lightgbm. Модель, полученная с помощью xgboost, оказалось самой медленной по скорости предсказаниям, по дргуим же критериям ее нельзя назватть несильно хорошей и несильно плохой.